In [56]:
from pyspark import SparkContext, SparkConf 
import pandas as pd
import re
from operator import add
import numpy as np

In [57]:
import os
os.environ["PYSPARK_PYTHON"]="/anaconda3/envs/tensorflow/bin/python3.5"
os.environ["PYTHONHASHSEED"]="123"
data1 = pd.read_csv("_Answers.csv",encoding="latin-1", iterator = True, chunksize=100)
# print(data1)
# data1.apply(lambda row: row.replace("\n", " "))
# data1.to_csv("aaa.csv",encoding='utf-8')
# print(data1.iloc[0])
hasHeader = False
os.remove("aaa.csv")
for chunk in data1:
    #print(chunk)
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace("\n", " "))
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace(",", ";"))
    #chunk["Title"] = chunk["Title"].map(lambda body: body.replace(",", ";"))
    #print(chunk)
    #print(chunk["Body"])
    if hasHeader:
        chunk.to_csv("aaa.csv", mode = "a", encoding="utf-8", header=False, index = False)
    else:
        chunk.to_csv("aaa.csv", mode = "a", encoding="utf-8", header=True, index = False)

    #break
#     print(type(chunk))
#     for index, row in chunk.iterrows():
#         print(row)
#         print(row["Body"])
#         chunk.iloc[index]["Body"] = chunk.iloc[index]["Body"].replace("\n", " ")
#         print(chunk.iloc[index]["Body"])
#         break
# data1.to_csv("aaa.csv",encoding='utf-8')
# data1 = pd.read_csv("aaa.csv")
# data1.replace('\n','')
# data1.to_csv("bbb.csv")
print("finish")
#df = spark.read.csv("aaa.csv",header=True);
#print(df.take(3))


AttributeError: 'float' object has no attribute 'replace'

In [92]:
sc = SparkContext.getOrCreate()

In [93]:
questionFile=sc.textFile("Questions.csv")
answerFile=sc.textFile("Answers.csv")
tagFile = sc.textFile("Tags.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

finished


In [94]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)

header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.take(2))

[['80', '26', '2008-08-01T13:57:07Z', '', '26', 'SQLStatement.execute() - multiple queries in one statement', '"<p>I\'ve written a database generation script in <a href=""http://en.wikipedia.org/wiki/SQL"">SQL</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime"">Adobe AIR</a> application:</p>  <pre><code>Create Table tRole (       roleID integer Primary Key       ;roleName varchar(40) ); Create Table tFile (     fileID integer Primary Key     ;fileName varchar(50)     ;fileDescription varchar(500)     ;thumbnailID integer     ;fileFormatID integer     ;categoryID integer     ;isFavorite boolean     ;dateAdded date     ;globalAccessCount integer     ;lastAccessTime date     ;downloadComplete boolean     ;isNew boolean     ;isSpotlight boolean     ;duration varchar(30) ); Create Table tCategory (     categoryID integer Primary Key     ;categoryName varchar(50)     ;parent_categoryID integer ); ... </code></pre>  <p>I execute this in Adobe AIR

In [65]:
stopWordsList = stopWordsFile.collect()
print(stopWordsList)

['a', 'The', 'One', 'Two', 'I', 'You', 'Any', 'Me', 'My', 'In', 'It', 'Is']


In [66]:
questionLower = question.map(lambda x: (x[0], (x[5]+x[6]).lower()))
print(questionLower.take(2))

[('80', 'sqlstatement.execute() - multiple queries in one statement"<p>i\'ve written a database generation script in <a href=""http://en.wikipedia.org/wiki/sql"">sql</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/adobe_integrated_runtime"">adobe air</a> application:</p>  <pre><code>create table trole (       roleid integer primary key       ;rolename varchar(40) ); create table tfile (     fileid integer primary key     ;filename varchar(50)     ;filedescription varchar(500)     ;thumbnailid integer     ;fileformatid integer     ;categoryid integer     ;isfavorite boolean     ;dateadded date     ;globalaccesscount integer     ;lastaccesstime date     ;downloadcomplete boolean     ;isnew boolean     ;isspotlight boolean     ;duration varchar(30) ); create table tcategory (     categoryid integer primary key     ;categoryname varchar(50)     ;parent_categoryid integer ); ... </code></pre>  <p>i execute this in adobe air using the following methods:</p>  <pre><cod

In [67]:
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
print(questionLower.take(2))

[('80', 'sqlstatement execute multiple queries in one statement p i ve written a database generation script in a href http en wikipedia org wiki sql sql a and want to execute it in my a href http en wikipedia org wiki adobe integrated runtime adobe air a application p pre code create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean duration varchar create table tcategory categoryid integer primary key categoryname varchar parent categoryid integer code pre p i execute this in adobe air using the following methods p pre code public static function runsqlfromfile filename string void var file file file applicationdirectory resolvepath filename var stream filestream new filestream stream open file fil

In [68]:
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.take(10))

[('80', 'sqlstatement'), ('80', 'execute'), ('80', 'multiple'), ('80', 'queries'), ('80', 'in'), ('80', 'one'), ('80', 'statement'), ('80', 'p'), ('80', 'i'), ('80', 've')]


In [69]:
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
print(questionPair.take(10))

[('80', 'sqlstatement'), ('80', 'execute'), ('80', 'multiple'), ('80', 'queries'), ('80', 'in'), ('80', 'one'), ('80', 'statement'), ('80', 'p'), ('80', 'i'), ('80', 've')]


In [70]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
print(questionPairfilter.take(10))

[('80', 'sqlstatement'), ('80', 'execute'), ('80', 'multiple'), ('80', 'queries'), ('80', 'in'), ('80', 'one'), ('80', 'statement'), ('80', 'p'), ('80', 'i'), ('80', 've')]


In [71]:
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
print(questionPairStage1.take(10))

[('80@lastaccesstime', 1), ('80@readutfbytes', 1), ('80@alert', 1), ('80@wiki', 2), ('90@red', 1), ('80@sqlerror', 1), ('90@tutorials', 2), ('80@create', 3), ('80@the', 4), ('90@s', 1)]


In [72]:
print(questionPairStage1.take(5))

[('80@lastaccesstime', 1), ('80@readutfbytes', 1), ('80@alert', 1), ('80@wiki', 2), ('90@red', 1)]


In [73]:
N = len(questionPair.countByKey())
print(N)

2


In [74]:
print(questionPairStage1.take(5))

[('80@lastaccesstime', 1), ('80@readutfbytes', 1), ('80@alert', 1), ('80@wiki', 2), ('90@red', 1)]


In [16]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage2MapPhase.take(5))

[('lastaccesstime', '80=1'), ('readutfbytes', '80=1'), ('alert', '80=1'), ('wiki', '80=2'), ('red', '90=1')]


In [17]:
stage2Map = questionPairStage2MapPhase.countByKey()
print("finish")

finish


In [18]:
print(stage2Map.get('statement'))

1


In [19]:
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.take(5))

[('lastaccesstime@80', 0.69314718055994529), ('readutfbytes@80', 0.69314718055994529), ('alert@80', 0.69314718055994529), ('wiki@80', 1.1736001944781467), ('red@90', 0.69314718055994529)]


In [20]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage3MapPhase.take(5))

[('80', 'lastaccesstime=0.69314718056'), ('80', 'readutfbytes=0.69314718056'), ('80', 'alert=0.69314718056'), ('80', 'wiki=1.17360019448'), ('90', 'red=0.69314718056')]


In [21]:
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.take(5))

[('80', 125.1906977852556), ('90', 16.976200374701293)]


In [22]:
print(questionPairStage3AggByKey.lookup('90'))

[]


In [23]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
print(questionPairStage3AggByKeysqr.lookup('90'))

[4.1202184862821651]


In [24]:
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

4.12021848628


In [25]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
print(questionPairStage3.take(500))

[('80', 'lastaccesstime=0.0619497318681'), ('80', 'readutfbytes=0.0619497318681'), ('80', 'alert=0.0619497318681'), ('80', 'wiki=0.104890013849'), ('90', 'red=0.168230685549'), ('80', 'sqlerror=0.0619497318681'), ('90', 'tutorials=0.284839310922'), ('80', 'create=0.130008468578'), ('80', 'the=0.0'), ('90', 's=0.168230685549'), ('90', 'client=0.168230685549'), ('80', 'only=0.104890013849'), ('80', 'at=0.0619497318681'), ('80', 'var=0.161653979002'), ('90', 'en=0.0'), ('80', 'new=0.130008468578'), ('90', 'branchmerge=0.168230685549'), ('80', 'air=0.104890013849'), ('80', 'error=0.104890013849'), ('80', 'globalaccesscount=0.0619497318681'), ('80', 'varchar=0.161653979002'), ('90', 'bean=0.168230685549'), ('80', 'generated=0.0619497318681'), ('80', 'fileid=0.0619497318681'), ('90', 'svn=0.168230685549'), ('90', 'explaining=0.168230685549'), ('80', 'void=0.104890013849'), ('80', 'sqlconnection=0.172948750559'), ('80', 'parent=0.0619497318681'), ('80', 'to=0.0'), ('80', 'execute=0.1478302958

In [26]:
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.take(5))

[('80', ['integer=0.19077057781', 'sqlstatement=0.182498343841', 'in=0.161653979002', 'pre=0.14783029583', 'boolean=0.14783029583', 'strsql=0.14783029583', 'stream=0.14783029583']), ('90', ['tutorials=0.284839310922', 'merging=0.284839310922', 'tortoisesvn=0.284839310922', 'good=0.284839310922', 'branching=0.284839310922', 'svn=0.168230685549', 'explaining=0.168230685549'])]


In [27]:
outMap=questionPairStage4.collectAsMap()
print(outMap)

{'90': ['tutorials=0.284839310922', 'merging=0.284839310922', 'tortoisesvn=0.284839310922', 'good=0.284839310922', 'branching=0.284839310922', 'svn=0.168230685549', 'explaining=0.168230685549'], '80': ['integer=0.19077057781', 'sqlstatement=0.182498343841', 'in=0.161653979002', 'pre=0.14783029583', 'boolean=0.14783029583', 'strsql=0.14783029583', 'stream=0.14783029583']}


In [28]:
tagWithHeader = tagFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = tagWithHeader.first() #extract header
tag = tagWithHeader.filter(lambda x: x != header)
print(tag.take(2))

[['80', 'flex'], ['80', 'actionscript-3']]


In [29]:
tagLower = tag.map(lambda x: (x[0], x[1].lower()))
print(tagLower.take(2))

[('80', 'flex'), ('80', 'actionscript-3')]


In [30]:
def f0(x): return re.sub('[^a-zA-Z]+', '', x)
tagLower = tagLower.mapValues(f0)
print(tagLower.take(2))

[('80', 'flex'), ('80', 'actionscript')]


In [31]:
tag_1000 = tagLower.map(lambda x: (x[0], x[1]+"=1000"))
print(tag_1000.take(2))

[('80', 'flex=1000'), ('80', 'actionscript=1000')]


In [32]:
questionPairStage5=questionPairStage4.map(lambda x:(x[0],x[1][0]+":"+x[1][1]+":"+x[1][2]+":"+x[1][3]+":"+x[1][4]+":"+x[1][5]+":"+x[1][6]) )
print(questionPairStage5.take(5))

[('80', 'integer=0.19077057781:sqlstatement=0.182498343841:in=0.161653979002:pre=0.14783029583:boolean=0.14783029583:strsql=0.14783029583:stream=0.14783029583'), ('90', 'tutorials=0.284839310922:merging=0.284839310922:tortoisesvn=0.284839310922:good=0.284839310922:branching=0.284839310922:svn=0.168230685549:explaining=0.168230685549')]


In [33]:
questionPairStage6=questionPairStage5.union(tag_1000)
def f6_1(a,b):
    return a+":"+b
def f6_2(a,b):
    return a+b
questionPairStage6=questionPairStage6.aggregateByKey("",f6_1,f6_2).map(lambda x: (x[0], x[1][1:]))
print(questionPairStage6.take(1))

[('80', 'integer=0.19077057781:sqlstatement=0.182498343841:in=0.161653979002:pre=0.14783029583:boolean=0.14783029583:strsql=0.14783029583:stream=0.14783029583:flex=1000:actionscript=1000:air=1000')]


In [34]:
def f7(x):
    listx= x[1].split(":")
    listx.sort(key=lambda item:float(item.split("=")[1]),reverse=True)
    output=""
    for i in range(num_keywords):
        output+=":"+listx[i]
    return (x[0],output)
questionPairStage7=questionPairStage6.map(f7).map(lambda x: (x[0], x[1][1:]))
print(questionPairStage7.take(2))

[('80', 'flex=1000:actionscript=1000:air=1000:integer=0.19077057781:sqlstatement=0.182498343841:in=0.161653979002:pre=0.14783029583'), ('90', 'svn=1000:tortoisesvn=1000:branch=1000:branchingandmerging=1000:tutorials=0.284839310922:merging=0.284839310922:tortoisesvn=0.284839310922')]


In [48]:
#Clustring
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
import itertools
from collections import defaultdict
from pyspark.mllib.linalg import Vectors
import os
os.environ['PYSPARK_PYTHON']="/anaconda3/envs/tensorflow/bin/python3.5"

sqlContext = SQLContext(sc)

def document_vector(document):
    id = document[1]
    counts = defaultdict(int)
    for token in document[0]:
        if token in vocabulary:
            token_id = vocabulary[token]
            counts[token_id] += 1
    counts = sorted(counts.items())
    keys = [x[0] for x in counts]
    values = [x[1] for x in counts]
    return (id, Vectors.sparse(len(vocabulary), keys, values))
#header = questionLower.first()


data=questionPairRaw#questionLower
documents=questionLower.map(lambda doc:[x for x in doc if len(x)>3])
#print(data.take(1))
#data = data.map(lambda words:(words[0],list(words[1].split(' '))))
#print(data.take(1))
data = data.map(lambda words: words[1])#.filter(lambda x: len(x)>3)  

#print(data.take(10))
data = data.map(lambda words: (words,1))
data = data.reduceByKey(add) 
#print(data.take(10))
data=data.map(lambda tuple: (tuple[1], tuple[0])).sortByKey(False)

vocabulary = data.map(lambda x: [x[0], x[1]]).map(lambda x: x[1]).zipWithIndex().collectAsMap()
inv_voc = {value: key for (key, value) in vocabulary.items()}
print(documents.take(5))
documents=documents.zipWithIndex().map(document_vector).map(list)
print(data.take(10))
print(vocabulary)
#all docs
num_topics=3
with open("output.txt", 'w') as f:
    lda_model = LDA.train(documents, k=num_topics, maxIterations=10)
                    
    topic_indices = lda_model.describeTopics(maxTermsPerTopic=10)
    
    # Output topics. Each is a distribution over words (matching word count vectors)
    print("Learned topics (as distributions over vocab of " + str(lda_model.vocabSize())\
      + " words):")
    topics = lda_model.topicsMatrix()
    for topic in range(3):
        print("Topic " + str(topic) + ":")
    for word in range(0, lda_model.vocabSize()):
        print(" " + str(topics[word][topic]))

        
    # Print topics, showing the top-weighted 10 terms for each topic
    for i in range(len(topic_indices)):
        f.write("Topic #{0}\n".format(i + 1))
        for j in range(len(topic_indices[i][0])):
            f.write("{0}\t{1}\n".format(inv_voc[topic_indices[i][0][j]] \
                .encode('utf-8'), topic_indices[i][1][j]))
            

    f.write("{0} topics distributed over {1} documents and {2} unique words\n"  \
        .format(num_topics, documents.count(), len(vocabulary)))
    


[['sqlstatement execute multiple queries in one statement p i ve written a database generation script in a href http en wikipedia org wiki sql sql a and want to execute it in my a href http en wikipedia org wiki adobe integrated runtime adobe air a application p pre code create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean duration varchar create table tcategory categoryid integer primary key categoryname varchar parent categoryid integer code pre p i execute this in adobe air using the following methods p pre code public static function runsqlfromfile filename string void var file file file applicationdirectory resolvepath filename var stream filestream new filestream stream open file filemode 

In [95]:

answerFileWithHeader = answerFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)

header = answerFileWithHeader.first() 
answer = answerFileWithHeader.filter(lambda x: x != header)
print(answer.take(2))

[['92', '61', '2008-08-01T14:45:37Z', '90', '13', '"<p><a href=""http://svnbook.red-bean.com/"">Version Control with Subversion</a></p>  <p>A very good resource for source control in general. Not really TortoiseSVN specific; though.</p>"', 'False'], ['124', '26', '2008-08-01T16:09:47Z', '80', '12', '"<p>I wound up using this. It is a kind of a hack; but it actually works pretty well. The only thing is you have to be very careful with your semicolons. : D</p>  <pre><code>var strSql:String = stream.readUTFBytes(stream.bytesAvailable);       var i:Number = 0; var strSqlSplit:Array = strSql.split("";""); for (i = 0; i &lt; strSqlSplit.length; i++){     NonQuery(strSqlSplit[i].toString()); } </code></pre> "', 'True']]


In [96]:
answer=answer.map(lambda x: (x[0],x[5]))
print(answer.take(2))


[('92', '"<p><a href=""http://svnbook.red-bean.com/"">Version Control with Subversion</a></p>  <p>A very good resource for source control in general. Not really TortoiseSVN specific; though.</p>"'), ('124', '"<p>I wound up using this. It is a kind of a hack; but it actually works pretty well. The only thing is you have to be very careful with your semicolons. : D</p>  <pre><code>var strSql:String = stream.readUTFBytes(stream.bytesAvailable);       var i:Number = 0; var strSqlSplit:Array = strSql.split("";""); for (i = 0; i &lt; strSqlSplit.length; i++){     NonQuery(strSqlSplit[i].toString()); } </code></pre> "')]


In [99]:
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
answerLower = answer.mapValues(f)
print(answerLower.take(2))

[('92', ' p a href http svnbook red bean com Version Control with Subversion a p p A very good resource for source control in general Not really TortoiseSVN specific though p '), ('124', ' p I wound up using this It is a kind of a hack but it actually works pretty well The only thing is you have to be very careful with your semicolons D p pre code var strSql String stream readUTFBytes stream bytesAvailable var i Number var strSqlSplit Array strSql split for i i lt strSqlSplit length i NonQuery strSqlSplit i toString code pre ')]


In [102]:
answerwithTag = answerLower.map(lambda x:(x[0],"Answer@"+x[0]+x[1]))
print(answerwithTag.take(2))

[('92', 'Answer@92 p a href http svnbook red bean com Version Control with Subversion a p p A very good resource for source control in general Not really TortoiseSVN specific though p '), ('124', 'Answer@124 p I wound up using this It is a kind of a hack but it actually works pretty well The only thing is you have to be very careful with your semicolons D p pre code var strSql String stream readUTFBytes stream bytesAvailable var i Number var strSqlSplit Array strSql split for i i lt strSqlSplit length i NonQuery strSqlSplit i toString code pre ')]


In [108]:
#(questionid, sum of answers)
def sumAsList(x):
    list=[]
    list.append(x[0])
    list.append(x[1])
    return list
answerSum=answerwithTag.reduceByKey(sumAsList)
print(answerSum.take(2))

[('92', 'Answer@92 p a href http svnbook red bean com Version Control with Subversion a p p A very good resource for source control in general Not really TortoiseSVN specific though p '), ('124', 'Answer@124 p I wound up using this It is a kind of a hack but it actually works pretty well The only thing is you have to be very careful with your semicolons D p pre code var strSql String stream readUTFBytes stream bytesAvailable var i Number var strSqlSplit Array strSql split for i i lt strSqlSplit length i NonQuery strSqlSplit i toString code pre ')]
